In [65]:
import wave
import cv2
import numpy as np

import random
import base64
import math
import os

Wc = np.indices((8, 8)).sum(axis=0) % 2


class Inserter:
    def __init__(self, file_dir, string_plain_text, key):
        image_file = File(file_dir)
#         string_plain_text = bytes(string_plain_text, 'utf-8')
        self.image_filesize = os.path.getsize(file_dir)
        self.ndarray = image_file.read_ndarray_image_file()
        self.h, self.w, self.color = self.ndarray.shape
        self.extension = "txt"
#         secret_message = File(secret_message_dir)
#         self.extension = secret_message.get_extention()
#         self.string_message = ""

        byte_message = bytes(string_plain_text, 'utf-8')
        self.message = base64.b64encode(byte_message).decode('utf-8')

        self.key = key

    def count_seed(self):
        return sum([ord(i) for i in self.key])

    def encrypt_message(self, encrypted, key):
        sign = 1 if encrypted else 0

        self.ndarray[0][0][0] = self.ndarray[0][0][0] & 254 | sign
        if encrypted:
            self.string_message = encrypt_vigenere(self.string_message, key)

    def random_list(self, randomize_frames):
        sign = 1 if randomize_frames else 0

        self.ndarray[0][0][1] = self.ndarray[0][0][1] & 254 | sign
        if randomize_frames:
            random.seed(self.seed)
            if self.method == 'lsb':
                random.shuffle(self.pixel_list)
            elif self.method == 'bpcs':
                random.shuffle(self.block_list)

    def modify_pixel(self, array_bit):
        index = 0
        for i in self.pixel_list:
            if index >= len(array_bit):
                break
            if i >= 3:
                h, w, color = self.get_ndarray_pos(i)
                self.ndarray[h][w][color] = self.ndarray[h][w][color] & 254 | array_bit[index]
                index += 1
        if index < len(array_bit):
            error = "Ukuran pesan melebihi kapasitas payload!"
            raise RuntimeError(error)

    def insert_alpha(self):
        alpha_str = str(self.alpha)[:7].ljust(7, '0')
        alpha_bits = list(map(int, ''.join([bin(ord(i)).lstrip('0b').rjust(8, '0') for i in alpha_str])))
        index = 0
        for i in range(1, 8):
            for j in range(8):
                self.ndarray[i][j][0] = self.ndarray[i][j][0] & 254 | alpha_bits[index]
                index += 1

    def pbc_to_cgc(self):
        b = self.ndarray
        g = b >> 7
        for i in range(7, 0, -1):
            g <<= 1
            g |= ((b >> i) & 1) ^ ((b >> (i - 1)) & 1)
        self.ndarray = g

    def cgc_to_pbc(self):
        g = self.ndarray
        b = g >> 7
        for i in range(7, 0, -1):
            b_before = b.copy()
            b <<= 1
            b |= (b_before & 1) ^ ((g >> (i - 1)) & 1)
        self.ndarray = b

    def complexity(self, matrix):
        maxim = ((matrix.shape[0] - 1) * matrix.shape[1]) + ((matrix.shape[1] - 1) * matrix.shape[0])
        curr = 0.0
        first = matrix[0,0]
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if (first != matrix[i,j]):
                    curr = curr + 1
                    first = matrix[i,j]
        first = matrix[0,0]
        for i in range(matrix.shape[1]):
            for j in range(matrix.shape[0]):
                if (first != matrix[j,i]):
                    curr = curr + 1
                    first = matrix[j,i]
        return curr/maxim
    
    def conjugate(self, P):
        return P ^ Wc

    def modify_block(self, array_bit):
        index = 0
        for bitplane in range(7, -1, -1):
            # bitplane 0 = msb, 7 = lsb
            for h, w, color in self.block_list:
                if index >= len(array_bit):
                    break
                if h != 0 and w != 0 and color != 0:
                    matrix = self.ndarray[h:h+8, w:w+8, color] >> (7 - bitplane) & 1
                    if self.complexity(matrix) > self.alpha:
                        current_bit = np.zeros((8, 8), dtype=int)
                        for i in range(8):
                            for j in range(8):
                                if (i > 0 or j > 0) and index < len(array_bit):
                                    current_bit[i, j] = array_bit[index]
                                    index += 1

                        if self.complexity(current_bit) <= self.alpha:
                            current_bit = self.conjugate(current_bit)
                            current_bit[0, 0] = 1

                        left = (self.ndarray[h:h+8, w:w+8, color] >> (8 - bitplane)) << (8 - bitplane)
                        right = self.ndarray[h:h+8, w:w+8, color] & ((1 << (8 - bitplane - 1)) - 1)
                        self.ndarray[h:h+8, w:w+8, color] = left + (current_bit << 7 - bitplane) + right
            if index >= len(array_bit):
                break
        if index < len(array_bit):
            error = "Ukuran pesan melebihi kapasitas payload!"
            raise RuntimeError(error)

    def get_ndarray_pos(self, idx):
        color = idx % self.color
        w = (idx // self.color) % self.w
        h = idx // (self.color * self.w)
        return h, w, color

    def insert_message(self, encrypted=False, randomize=False, method='bpcs', alpha=0.3):
        self.seed = self.count_seed()
        self.method = method
        self.alpha = alpha

        self.string_message = str(len(self.message)) + '#' + "txt" + '#' + self.message
        self.encrypt_message(encrypted, self.key)

        bits = map(int, ''.join(
            [bin(ord(i)).lstrip('0b').rjust(8, '0') for i in self.string_message]))
        array_bit = list(bits)

        if method == 'lsb':
            self.pixel_list = list(range(self.h * self.w * self.color))
            self.random_list(randomize)
            self.modify_pixel(array_bit)
            self.ndarray[0, 0, 2] = self.ndarray[0, 0, 2] & 254
        elif method == 'bpcs':
            self.block_list = []
            for h in range(0, self.h - (self.h % 8), 8):
                for w in range(0, self.w - (self.w % 8), 8):
                    for color in range(0, self.color):
                        self.block_list += [(h, w, color)]
            self.random_list(randomize)
            self.pbc_to_cgc()
            self.modify_block(array_bit)
            self.cgc_to_pbc()
            self.insert_alpha()
            self.ndarray[0, 0, 2] = self.ndarray[0, 0, 2] & 254 | 1

        return self.ndarray

class Extractor:
    def __init__(self, file_dir, key):
        stegano_image_file = File(file_dir)
        self.ndarray = stegano_image_file.read_ndarray_image_file()
        self.h, self.w, self.color = self.ndarray.shape
        self.key = key

    def conjugate(self, P):
        return P ^ Wc

    def pbc_to_cgc(self):
        b = self.ndarray
        g = b >> 7
        for i in range(7, 0, -1):
            g <<= 1
            g |= ((b >> i) & 1) ^ ((b >> (i - 1)) & 1)
        self.ndarray = g

    def cgc_to_pbc(self):
        g = self.ndarray
        b = g >> 7
        for i in range(7, 0, -1):
            b_before = b.copy()
            b <<= 1
            b |= (b_before & 1) ^ ((g >> (i - 1)) & 1)
        self.ndarray = b

    def complexity(self, matrix):
        maxim = ((matrix.shape[0] - 1) * matrix.shape[1]) + ((matrix.shape[1] - 1) * matrix.shape[0])
        curr = 0.0
        first = matrix[0,0]
        for i in range(matrix.shape[0]):
            for j in range(matrix.shape[1]):
                if (first != matrix[i,j]):
                    curr = curr + 1
                    first = matrix[i,j]
        first = matrix[0,0]
        for i in range(matrix.shape[1]):
            for j in range(matrix.shape[0]):
                if (first != matrix[j,i]):
                    curr = curr + 1
                    first = matrix[j,i]
        return curr/maxim

    def count_seed(self):
        return sum([ord(i) for i in self.key])

    def get_ndarray_pos(self, idx):
        color = idx % self.color
        w = (idx // self.color) % self.w
        h = idx // (self.color * self.w)
        return h, w, color

    def extract_alpha(self):
        index = 0
        mod_index = 8
        temp = ""
        alpha_str = ""

        for i in range(1, 8):
            for j in range(8):
                extracted = self.ndarray[i][j][0] & 1
                if index % mod_index != (mod_index - 1):
                    temp += str(extracted)
                else:
                    temp += str(extracted)
                    alpha_str += chr(int(temp, 2))
                    temp = ""
                index += 1
        print(alpha_str)
        return float(alpha_str)

    def extract_messages(self):
        self.seed = self.count_seed()

        extracted = [self.ndarray[self.get_ndarray_pos(i)] & 1 for i in range(self.h * self.w * self.color)]
        encrypted = extracted[0]
        random_pixels = extracted[1]
        method = 'bpcs' if extracted[2] else 'lsb'

        index = 0
        mod_index = 8

        message = ""
        temp = ""

        if method == 'lsb':
            pixel_list = list(range(len(extracted)))
            if random_pixels:
                random.seed(self.seed)
                random.shuffle(pixel_list)

            for i in pixel_list:
                if i >= 3:
                    if index % mod_index != (mod_index - 1):
                        temp += str(extracted[i])
                    else:
                        temp += str(extracted[i])
                        message += chr(int(temp, 2))
                        temp = ""
                    index += 1
        elif method == 'bpcs':
            alpha = self.extract_alpha()
            block_list = []
            for h in range(0, self.h - (self.h % 8), 8):
                for w in range(0, self.w - (self.w % 8), 8):
                    for color in range(0, self.color):
                        block_list += [(h, w, color)]
            if random_pixels:
                random.seed(self.seed)
                random.shuffle(block_list)
            self.pbc_to_cgc()
            for bitplane in range(7, -1, -1):
                for h, w, color in block_list:
                    if h != 0 and w != 0 and color != 0:
                        matrix = self.ndarray[h:h+8, w:w+8, color] >> (7 - bitplane) & 1
                        if self.complexity(matrix) > alpha:
                            if matrix[0, 0]:
                                matrix = self.conjugate(matrix)
                            for i in range(8):
                                for j in range(8):
                                    if i > 0 or j > 0:
                                        extracted_bit = matrix[i][j]
                                        if index % mod_index != (mod_index - 1):
                                            temp += str(extracted_bit)
                                        else:
                                            temp += str(extracted_bit)
                                            message += chr(int(temp, 2))
                                            temp = ""
                                        index += 1

        if encrypted:
            self.string_message = decrypt_vigenere(message, self.key)
        else:
            self.string_message = message

    def parse_message(self):
        message_info = self.string_message.split("#")
#         print("PARSE MESSAGE", message_info)
        self.len_message = int(message_info[0])
        self.extension = message_info[1]
        
    def extract_secret_message_to_text(self):
#         self.extract_messages()
#         self.parse_message()
        
        init = len(str(self.len_message)) + len(str(self.extension)) + 2
        decoded = self.string_message[init : init + self.len_message]
        bytes_file = decoded.encode('utf-8')
#         print(bytes_file.encode('utf-8'))
        bytes_file = base64.b64decode(bytes_file)
#         print("Bytes File\n",bytes_file)
        return bytes_file
#         print(decoded)

    def write_secret_message(self):
        init = len(str(self.len_message)) + len(str(self.extension)) + 2
        decoded = self.string_message[init : init + self.len_message]

        bytes_file = decoded.encode('utf-8')
#         print(bytes_file.encode('utf-8'))
        bytes_file = base64.b64decode(bytes_file)
        
        print("BYTE FILE NIH\n", bytes_file)
        print("Selesai")
        return bytes_file



class File:
    def __init__(self, filename):
        self.filename = filename
#         self.file_size = os.path.getsize(file_path)

    def read_ndarray_image_file(self):
        return cv2.imread(self.filename)

    def write_image_file(self, image):
        cv2.imwrite(self.filename, image)

    def read_frame_audio_file(self):
        song = wave.open(self.filename, mode='rb')
        frame_bytes = bytearray(list(song.readframes(song.getnframes())))
        song.close()

        return frame_bytes

    def init_buff_audio_file(self):
        song = wave.open(self.filename, mode='rb')
        init_buff = song.readframes(-1)
        init_buff = [item + 0 for item in init_buff]
        song.close()

        return init_buff

    def write_audio_file(self, frame, params):
        song = wave.open(self.filename, mode='wb')
        song.setparams(params)
        song.writeframes(frame)
        song.close()

    def get_params_audio(self):
        song = wave.open(self.filename, mode='rb')
        params = song.getparams()
        song.close()

        return params

    def get_extention(self):
        has_extention = self.filename.split("/")[-1].find(".") != -1
        self.extension = ""

        if (has_extention):
            self.extension = self.filename.split(".")[-1]

        return self.extension

    def read_files(self):
        with open(self.filename, "rb") as f:
            byte_file = f.read()

        return byte_file

    def write_files(self, bytes_file):
        with open(self.filename, 'wb') as fd:
            print(bytes_file)
            fd.write(bytes_file)

            
def encrypt_vigenere(plainText, key):
    output = ""
    key = key.upper()

    for i in range(0, len(plainText)):
        output += chr((ord(plainText[i]) + ord(key[i % len(key)])) % 256)

    return output


def decrypt_vigenere(cipherText, key):
    output = ""
    key = key.upper()

    for i in range(0, len(cipherText)):
        output += chr((ord(cipherText[i]) - ord(key[i % len(key)])) % 256)

    return output


## Video to Frames Extractor

In [2]:
# Opens the Video file
cap= cv2.VideoCapture('sample/video/alienoid-trailer.mp4')
fps = cap.get(cv2.CAP_PROP_FPS)
width = cap.get(cv2.CAP_PROP_FRAME_WIDTH )
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT )
print("FPS:", fps)
print("HEIGHT:", height)
print("WIDTH:", width)
i=0
total_frames = 0
while(cap.isOpened()):
    
    ret, frame = cap.read()
    if ret == False:
        total_frames = i
        break
    cv2.imwrite('frames/alienoid-trailer' + str(i) +'.png',frame)
    print(f"extract frame ke-{i}")
    i+=1

cap.release()
cv2.destroyAllWindows()

FPS: 23.976023976023978
HEIGHT: 720.0
WIDTH: 1280.0
extract frame ke-0
extract frame ke-1
extract frame ke-2
extract frame ke-3
extract frame ke-4
extract frame ke-5
extract frame ke-6
extract frame ke-7
extract frame ke-8
extract frame ke-9
extract frame ke-10
extract frame ke-11
extract frame ke-12
extract frame ke-13
extract frame ke-14
extract frame ke-15
extract frame ke-16
extract frame ke-17
extract frame ke-18
extract frame ke-19
extract frame ke-20
extract frame ke-21
extract frame ke-22
extract frame ke-23
extract frame ke-24
extract frame ke-25
extract frame ke-26
extract frame ke-27
extract frame ke-28
extract frame ke-29
extract frame ke-30
extract frame ke-31
extract frame ke-32
extract frame ke-33
extract frame ke-34
extract frame ke-35
extract frame ke-36
extract frame ke-37
extract frame ke-38
extract frame ke-39
extract frame ke-40
extract frame ke-41
extract frame ke-42
extract frame ke-43
extract frame ke-44
extract frame ke-45
extract frame ke-46
extract frame ke-4

extract frame ke-398
extract frame ke-399
extract frame ke-400
extract frame ke-401
extract frame ke-402
extract frame ke-403
extract frame ke-404
extract frame ke-405
extract frame ke-406
extract frame ke-407
extract frame ke-408
extract frame ke-409
extract frame ke-410
extract frame ke-411
extract frame ke-412
extract frame ke-413
extract frame ke-414
extract frame ke-415
extract frame ke-416
extract frame ke-417
extract frame ke-418
extract frame ke-419
extract frame ke-420
extract frame ke-421
extract frame ke-422
extract frame ke-423
extract frame ke-424
extract frame ke-425
extract frame ke-426
extract frame ke-427
extract frame ke-428
extract frame ke-429
extract frame ke-430
extract frame ke-431
extract frame ke-432
extract frame ke-433
extract frame ke-434
extract frame ke-435
extract frame ke-436
extract frame ke-437
extract frame ke-438
extract frame ke-439
extract frame ke-440
extract frame ke-441
extract frame ke-442
extract frame ke-443
extract frame ke-444
extract frame

extract frame ke-793
extract frame ke-794
extract frame ke-795
extract frame ke-796
extract frame ke-797
extract frame ke-798
extract frame ke-799
extract frame ke-800
extract frame ke-801
extract frame ke-802
extract frame ke-803
extract frame ke-804
extract frame ke-805
extract frame ke-806
extract frame ke-807
extract frame ke-808
extract frame ke-809
extract frame ke-810
extract frame ke-811
extract frame ke-812
extract frame ke-813
extract frame ke-814
extract frame ke-815
extract frame ke-816
extract frame ke-817
extract frame ke-818
extract frame ke-819
extract frame ke-820
extract frame ke-821
extract frame ke-822
extract frame ke-823
extract frame ke-824
extract frame ke-825
extract frame ke-826
extract frame ke-827
extract frame ke-828
extract frame ke-829
extract frame ke-830
extract frame ke-831
extract frame ke-832
extract frame ke-833
extract frame ke-834
extract frame ke-835
extract frame ke-836
extract frame ke-837
extract frame ke-838
extract frame ke-839
extract frame

extract frame ke-1176
extract frame ke-1177
extract frame ke-1178
extract frame ke-1179
extract frame ke-1180
extract frame ke-1181
extract frame ke-1182
extract frame ke-1183
extract frame ke-1184
extract frame ke-1185
extract frame ke-1186
extract frame ke-1187
extract frame ke-1188
extract frame ke-1189
extract frame ke-1190
extract frame ke-1191
extract frame ke-1192
extract frame ke-1193
extract frame ke-1194
extract frame ke-1195
extract frame ke-1196
extract frame ke-1197
extract frame ke-1198
extract frame ke-1199
extract frame ke-1200
extract frame ke-1201
extract frame ke-1202
extract frame ke-1203
extract frame ke-1204
extract frame ke-1205
extract frame ke-1206
extract frame ke-1207
extract frame ke-1208
extract frame ke-1209
extract frame ke-1210
extract frame ke-1211
extract frame ke-1212
extract frame ke-1213
extract frame ke-1214
extract frame ke-1215
extract frame ke-1216
extract frame ke-1217
extract frame ke-1218
extract frame ke-1219
extract frame ke-1220
extract fr

extract frame ke-1557
extract frame ke-1558
extract frame ke-1559
extract frame ke-1560
extract frame ke-1561
extract frame ke-1562
extract frame ke-1563
extract frame ke-1564
extract frame ke-1565
extract frame ke-1566
extract frame ke-1567
extract frame ke-1568
extract frame ke-1569
extract frame ke-1570
extract frame ke-1571
extract frame ke-1572
extract frame ke-1573
extract frame ke-1574
extract frame ke-1575
extract frame ke-1576
extract frame ke-1577
extract frame ke-1578
extract frame ke-1579
extract frame ke-1580
extract frame ke-1581
extract frame ke-1582
extract frame ke-1583
extract frame ke-1584
extract frame ke-1585
extract frame ke-1586
extract frame ke-1587
extract frame ke-1588
extract frame ke-1589
extract frame ke-1590
extract frame ke-1591
extract frame ke-1592
extract frame ke-1593
extract frame ke-1594
extract frame ke-1595
extract frame ke-1596
extract frame ke-1597
extract frame ke-1598
extract frame ke-1599
extract frame ke-1600
extract frame ke-1601
extract fr

In [ ]:
dir_pesan = "pesan.txt"
dir_gambar = "alienoid.png"

In [ ]:
total_frames

In [ ]:
# numpy image rgb
instr = Inserter(dir_gambar, dir_pesan, "bebas")
np_image = instr.insert_message(encrypted=True, randomize=True, alpha=0.3)

out_filename = "alienoid_stego_image_alpha0.3.png"
outfile = File(out_filename)
outfile.write_image_file(np_image)

## Decrypt

In [61]:
dir_gambar_stego_image = "frames_stego/alienoid_stego_image0.png"
key = "bebas"
extrctr = Extractor(dir_gambar_stego_image, key)
extrctr.extract_messages()
extrctr.parse_message()
extrctr.extract_secret_message_to_text()
# byte = extrctr.write_secret_message()
# output = 'pesan_extracted.txt'
# output_file = File(output)
# output_file.write_files(byte)

0.30000
Bytes File
 b'UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONE'


b'UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONE'

## Read Text File

In [ ]:
f = open("pesan.txt", 'r')
characters = f.read()
f.close()
characters

In [ ]:
def split_all_characters(characters):
    return list(characters)

In [56]:
f = open("pesan.txt", 'r')
plaintext = f.read()
f.close
print(plaintext)

UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA
TAHUN 1945
PEMBUKAAN
( P r e a m b u l e )
 Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,
maka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.
 Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat
yang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan
pintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil
dan makmur.
 Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan
luhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan
dengan ini kemerdekaannya.
 Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang
melindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk
memajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut
melaksanakan ketertiban dunia yang berdasarkan ke

In [57]:
# def generate_text_parts(dir_pesan):
#     pesan = open(dir_pesan, 'r')
#     characters = pesan.read()
#     pesan.close()
#     batch_message = split_string(characters, total_frames)
#     for i in range(total_frames):
#         new_text_file = open(f"message_parts/pesan{i}.txt", 'w')
#         new_text_file.write(batch_message[i])
#         new_text_file.close()
#         print(f"Writing files ...{i}")
        

def batch_inserter(string_plaintext):
    plaintext = list(string_plaintext)
    i = 0
    char_idx = 0
    while len(plaintext) > 0:
        dir_gambar = f"frames/alienoid-trailer{i}.png"
        img_filesize = os.path.getsize(dir_gambar)
        total_input_plaintext_chars = img_filesize // (1024 * 4) # 25% dari total file size
        print(f"""
            Urutan       : {i}
            Ukuran File  : {img_filesize / 1024}
            Max Char     : {total_input_plaintext_chars}
            Len Plaintext: {len(plaintext)}
            Text         :\n {''.join(plaintext[:total_input_plaintext_chars])}  
        """)
        input_plaintext = ''.join(plaintext[:total_input_plaintext_chars])
        instr = Inserter(dir_gambar, input_plaintext, "bebas")
        np_image = instr.insert_message(encrypted=True, randomize=True, alpha=0.3)

        out_filename = f"frames_stego/alienoid_stego_image{i}.png"
        outfile = File(out_filename)
        outfile.write_image_file(np_image)
        print(f"inserting message into frame-{i}")
        
        plaintext = plaintext[total_input_plaintext_chars:]
        i += 1
#         input_plaintext = plaintext[:total_input_plaintext_chars]
#         plaintext = plaintext[total_input_plaintext_chars:]
#         instr = Inserter(dir_gambar, input_plaintext, "bebas")
#         np_image = instr.insert_message(encrypted=True, randomize=True, alpha=0.3)

#         out_filename = f"frames_stego/alienoid_stego_image{i}.png"
#         outfile = File(out_filename)
#         outfile.write_image_file(np_image)
#         print(f"inserting message into frame-{i}")

In [52]:
batch_inserter(plaintext)


            Urutan       : 0
            Ukuran File  : 170.3642578125
            Max Char     : 42
            Len Plaintext: 12320
            Text         :
 UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONE  
        
inserting message into frame-0

            Urutan       : 1
            Ukuran File  : 177.9150390625
            Max Char     : 44
            Len Plaintext: 12278
            Text         :
 SIA
TAHUN 1945
PEMBUKAAN
( P r e a m b u l e  
        
inserting message into frame-1

            Urutan       : 2
            Ukuran File  : 180.5576171875
            Max Char     : 45
            Len Plaintext: 12234
            Text         :
  )
 Bahwa sesungguhnya kemerdekaan itu ialah   
        
inserting message into frame-2

            Urutan       : 3
            Ukuran File  : 181.5966796875
            Max Char     : 45
            Len Plaintext: 12189
            Text         :
 hak segala bangsa dan oleh sebab itu,
maka pe  
        
inserting message into frame-3


inserting message into frame-32

            Urutan       : 33
            Ukuran File  : 146.4306640625
            Max Char     : 36
            Len Plaintext: 10808
            Text         :
 
--------------------------
UNDANG-U  
        
inserting message into frame-33

            Urutan       : 34
            Ukuran File  : 133.7060546875
            Max Char     : 33
            Len Plaintext: 10772
            Text         :
 NDANG DASAR NEGARA REPUBLIK INDON  
        
inserting message into frame-34

            Urutan       : 35
            Ukuran File  : 133.6591796875
            Max Char     : 33
            Len Plaintext: 10739
            Text         :
 ESIA
TAHUN 1945
PEMBUKAAN
( P r e  
        
inserting message into frame-35

            Urutan       : 36
            Ukuran File  : 127.3935546875
            Max Char     : 31
            Len Plaintext: 10706
            Text         :
  a m b u l e )
 Bahwa sesungguh  
        
inserting message into frame-36

  

inserting message into frame-67

            Urutan       : 68
            Ukuran File  : 108.9052734375
            Max Char     : 27
            Len Plaintext: 9802
            Text         :
 ia yang berdasarkan kemerde  
        
inserting message into frame-68

            Urutan       : 69
            Ukuran File  : 108.9716796875
            Max Char     : 27
            Len Plaintext: 9775
            Text         :
 kaan, perdamaian abadi dan
  
        
inserting message into frame-69

            Urutan       : 70
            Ukuran File  : 108.9990234375
            Max Char     : 27
            Len Plaintext: 9748
            Text         :
 keadilan sosial, maka disus  
        
inserting message into frame-70

            Urutan       : 71
            Ukuran File  : 108.8935546875
            Max Char     : 27
            Len Plaintext: 9721
            Text         :
 unlah Kemerdekaan Kebangsaa  
        
inserting message into frame-71

            Urutan       : 72
 

inserting message into frame-96

            Urutan       : 97
            Ukuran File  : 309.8564453125
            Max Char     : 77
            Len Plaintext: 7373
            Text         :
  telah sampailah kepada saat
yang berbahagia dengan selamat sentausa menganta  
        
inserting message into frame-97

            Urutan       : 98
            Ukuran File  : 297.283203125
            Max Char     : 74
            Len Plaintext: 7296
            Text         :
 rkan rakyat Indonesia ke depan
pintu gerbang kemerdekaan Negara Indonesia,  
        
inserting message into frame-98

            Urutan       : 99
            Ukuran File  : 287.173828125
            Max Char     : 71
            Len Plaintext: 7222
            Text         :
  yang merdeka, bersatu, berdaulat, adil
dan makmur.
 Atas berkat rakhma  
        
inserting message into frame-99

            Urutan       : 100
            Ukuran File  : 277.6201171875
            Max Char     : 69
            Len Plainte

inserting message into frame-127

            Urutan       : 128
            Ukuran File  : 183.8203125
            Max Char     : 45
            Len Plaintext: 5593
            Text         :
 Kuasa dan dengan didorongkan oleh keinginan
l  
        
inserting message into frame-128

            Urutan       : 129
            Ukuran File  : 181.1181640625
            Max Char     : 45
            Len Plaintext: 5548
            Text         :
 uhur, supaya berkehidupan kebangsaan yang beb  
        
inserting message into frame-129

            Urutan       : 130
            Ukuran File  : 184.8349609375
            Max Char     : 46
            Len Plaintext: 5503
            Text         :
 as, maka rakyat Indonesia menyatakan
dengan in  
        
inserting message into frame-130

            Urutan       : 131
            Ukuran File  : 185.9091796875
            Max Char     : 46
            Len Plaintext: 5457
            Text         :
 i kemerdekaannya.
 Kemudian dari pada itu u

inserting message into frame-161

            Urutan       : 162
            Ukuran File  : 95.2080078125
            Max Char     : 23
            Len Plaintext: 4335
            Text         :
 erjuangan pergerakan ke  
        
inserting message into frame-162

            Urutan       : 163
            Ukuran File  : 91.2294921875
            Max Char     : 22
            Len Plaintext: 4312
            Text         :
 merdekaan Indonesia te  
        
inserting message into frame-163

            Urutan       : 164
            Ukuran File  : 87.943359375
            Max Char     : 21
            Len Plaintext: 4290
            Text         :
 lah sampailah kepada   
        
inserting message into frame-164

            Urutan       : 165
            Ukuran File  : 77.6220703125
            Max Char     : 19
            Len Plaintext: 4269
            Text         :
 saat
yang berbahagi  
        
inserting message into frame-165

            Urutan       : 166
            Ukuran 

RuntimeError: Ukuran pesan melebihi kapasitas payload!

## Frames to Video

In [ ]:
 # choose codec according to format needed

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
video = cv2.VideoWriter('alienoid-trailer-stego-vid.mp4', fourcc, frameSize = (int(width), int(height)), fps=int(fps))

for j in range(total_frames):
    path = 'frames_stego/alienoid_stego_image' + str(j) + '.png'
    img = cv2.imread(path)
    video.write(img)
    print(f"Menjahit frame jadi video {j}")

cv2.destroyAllWindows()
video.release()

## Stego Frames to Plaintext

In [ ]:
# result = ""
# for i in range(total_frames):
#     print(f"Plaintext ke-{i}")
#     dir_gambar_stego_image = f"frames_stego/alienoid_stego_image{i}.png"
#     key = "bebas"
#     extrctr = Extractor(dir_gambar_stego_image, key)
#     extrctr.extract_messages()
#     extrctr.parse_message()
#     result += extrctr.extract_secret_message_to_text()
#     byte = extrctr.write_secret_message()
#     output = 'pesan_extracted.txt'
#     output_file = File(output)
#     output_file.write_files(byte)

In [58]:
APP_FOLDER = 'frames_stego'
total_files = 0
total_dir = 0
for base, dirs, files in os.walk(APP_FOLDER):
    print('Searching in : ',base)
    for directories in dirs:
        total_dir += 1
    for Files in files:
        total_files += 1
        
print(f"""
Total Files   : {total_files}
Total Dirs    : {total_dir}
""")

Searching in :  frames_stego

Total Files   : 193
Total Dirs    : 0



In [64]:
# dir_gambar_stego_image = f"alienoid_stego_image.png"
result = ""
key = "bebas"
for i in range(total_files):
    print(f"extracting files....{i}")
    extrctr = Extractor(f"frames_stego/alienoid_stego_image{i}.png", key)
    extrctr.extract_messages()
    extrctr.parse_message()
    result += str(extrctr.extract_secret_message_to_text())[2:-1]
    print("Result\n", result)

print(result)

extracting files....0
0.30000
Bytes File
 b'UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONE'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONE
extracting files....1
0.30000
Bytes File
 b'SIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e
extracting files....2
0.30000
Bytes File
 b' )\n Bahwa sesungguhnya kemerdekaan itu ialah '
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah 
extracting files....3
0.30000
Bytes File
 b'hak segala bangsa dan oleh sebab itu,\nmaka pe'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka pe
extracting files....4
0.30000
Bytes File
 b'njajahan di atas dunia harus dihapuskan, karen'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\

0.30000
Bytes File
 b'ang\nmelindungi segenap bangsa Indonesia dan selu'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan selu
extracting files....18
0.30000
Bytes File
 b'ruh tumpah darah Indone

0.30000
Bytes File
 b'alam suatu susunan Negara\nRepublik Indonesia y'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, men

0.30000
Bytes File
 b'ta dengan mewujudkan suatu Keadilan sosi'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdask

0.30000
Bytes File
 b' a m b u l e )\n Bahwa sesungguh'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehid

...

0.30000
Bytes File
 b'donesia me'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut\

0.30000
Bytes File
 b'nyatakan\nd'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut

0.30000
Bytes File
 b'engan ini '
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut\

0.30000
Bytes File
 b'kemerdekaanny'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ik

0.30000
Bytes File
 b'a.\n Kemudian da'
UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan

In [67]:
print(result)

UNDANG-UNDANG DASAR NEGARA REPUBLIK INDONESIA\nTAHUN 1945\nPEMBUKAAN\n( P r e a m b u l e )\n Bahwa sesungguhnya kemerdekaan itu ialah hak segala bangsa dan oleh sebab itu,\nmaka penjajahan di atas dunia harus dihapuskan, karena tidak sesuai dengan perikemanusiaan dan peri-keadilan.\n Dan perjuangan pergerakan kemerdekaan Indonesia telah sampailah kepada saat\nyang berbahagia dengan selamat sentausa mengantarkan rakyat Indonesia ke depan\npintu gerbang kemerdekaan Negara Indonesia, yang merdeka, bersatu, berdaulat, adil\ndan makmur.\n Atas berkat rakhmat Allah Yang Maha Kuasa dan dengan didorongkan oleh keinginan\nluhur, supaya berkehidupan kebangsaan yang bebas, maka rakyat Indonesia menyatakan\ndengan ini kemerdekaannya.\n Kemudian dari pada itu untuk membentuk suatu Pemerintah Negara Indonesia yang\nmelindungi segenap bangsa Indonesia dan seluruh tumpah darah Indonesia dan untuk\nmemajukan kesejahteraan umum, mencerdaskan kehidupan bangsa, dan ikut\nmelaksanakan ketertiban dunia yan